# Libaries and definitions

In [ ]:
# Import libaries

import populartimes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
import os
import conda
import imageio
from natsort import natsorted, ns

# work around to import basemap 
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon

    # Define variables

# Project directory & name
pdir = "/home/ubuntu/Documents/Sound_Diplomacy/Projects"
pname = "berlin"

# Popular times
api_key = "
types = ["bar"] # https://developers.google.com/places/supported_types
p1 = [52.514514, 13.460177]  # lower bound - lat/lng of point delimiting the search are
p2 = [52.512791, 13.458323]  # upper bound - lat/lng of point delimiting the search are

# Colors

sdcol = '#23b5b5'

# Basemap
width = 52000
height = 42000
resol = 'i'
proj = 'tmerc'
lat0 = 52.511677
lon0 = 13.401777

fillcol = 'aqua'
lakecol = 'aqua'
contcol = 'white'

# Gif
my_dpi=300
mcol = sdcol

# Functions

In [ ]:
# flattens coordinate dictionary, creates daytime (e.g. Mon23)

def edit(lst):
    
    n = len(lst)
    
    for place in range(0,n):
        
        # get latitude and longitude from coordinates
        lat_dict = {'lat': lst[place]['coordinates']['lat']}
        lng_dict = {'lng': lst[place]['coordinates']['lng']}
            
        # add to list
        lst[place].update(lat_dict)
        lst[place].update(lng_dict)
        
        # create and add daytime
        for day in range(0,7):
            for time in range(0,24):

                day_str = lst[place]['populartimes'][day]['name'][:3] # get first three letter of the weekday
                daytime_dict = {day_str + str(time): lst[place]['populartimes'][day]['data'][time]} # weekday + daytime
                lst[place].update(daytime_dict)
            
    return lst

# creates data frame from list, edits data frame, returns it and exports it to csv

def process(lst, pnam):
    
    edit(lst)
    
    # turn to data frame
    df = pd.DataFrame(lst)    
    
    # drop dictonaries, reshape to panel data
    df = df.drop(columns=['populartimes', 'coordinates'])
    df = df.melt(id_vars=['name','rating','rating_n','time_spent','types','address','id','lat','lng'])
    
    # day variable
    df['day'] = df['variable'].str.extract('([A-Z]\w\w)', expand=True)
    days = {'Mon': 1,'Tue': 2,'Wed': 3,'Thu': 4,'Fri': 5,'Sat': 6,'Sun': 7}
    df.day = [days[item] for item in df.day]
    
    # day time variable
    df['time'] = df['variable'].str.extract('([0-9]+)', expand=True)
    df['time'] = df['time'].astype(str).astype(int)
    
    # drop obsolete columns, sort by name weekday and day time
    #df = df.drop(columns=['variable'])
    df = df.rename(index=str, columns={"variable": "daytime"})
    df = df.sort_values(by=['name','day','time'])
    
    os.chdir(pdir+"/Populartimes/data")
    df.to_csv(pnam+".csv", index=False)
    
    return df

def plotgif(df):

    daytim = df.drop_duplicates('daytime')['daytime']
    poptim = df.sort_values(by=['day','time','name']) # important!

    df.set_index("daytime", inplace=True)
    df.head()

    lat = np.array(df.loc['Mon0'].lat)
    lng = np.array(df.loc['Mon0'].lng)

    values = np.array(df.value)

    npla = len(lat) # number of places (counted by different latitudes)
    nval = len(values) # number of values
    nimg = int(nval/npla) # number of images

    msizes = {} # dictionary for marker sizes

    for no, i in enumerate(range(0,nval,npla)):
        j=i+npla
        msizes["msize{0}".format(no)] = 1+1.02*values[i:j]

    os.chdir(pdir+"/Populartimes/graphs/gif/files")

    for i in range(0,nimg):
        msize = msizes['msize'+str(i)]
        fig = plt.figure(figsize=(480/my_dpi, 480/my_dpi), dpi=my_dpi)
        ax = fig.gca()
        _ = ax.set(xlim=(52.512,52.515), ylim=(13.458,13.461))
        _ = ax.axis('off')
        _ = ax.set_title(daytim[i], fontsize=5)
        _ = ax.scatter(lat, lng, color=mcol, s=msize)
        filename = 'img'+str(i)+'.png'
        plt.savefig(filename, dpi=my_dpi)
        plt.clf()
        
    png_dir= pdir+"/Populartimes/graphs/gif/files"
    images=[]
    for file_name in natsorted(os.listdir(png_dir)):
        if file_name.endswith('.png'):
            file_path = os.path.join(png_dir, file_name)
            images.append(imageio.imread(file_path))

    os.chdir(pdir+"/Populartimes/graphs/gif")
    imageio.mimsave('populartimes_'+str(pname)+'.gif', images, duration=0.15)
    
def plotmarks(dta):
    lng = np.array(dta.lng)
    lat = np.array(dta.lat)
    val = np.array(dta.value)
    x, y = map(lng, lat)
    msize = 1 + val
    map.plot(x, y, 'o',markersize=msize, color=mcol, alpha=0.8) # what does alpha do?


# Get popular times data

In [ ]:
poptim_rawdata = populartimes.get(api_key, types, p1, p2)

# Process popular time data

In [ ]:
poptim = process(poptim_rawdata, pname)

# Map

In [ ]:
width = 500
height = 420
resol = 'i'
proj = 'tmerc'
lat0 = 52.5134798
lon0 = 13.4588845

# Draw basemap

os.chdir(pdir+"/Populartimes/shapefiles")
#os.chdir(pdir+"/Populartimes/shapefiles/openstreetmap")

map = Basemap(width=width, height=height, resolution=resol, projection=proj, lat_0 = lat0, lon_0 = lon0)
map.drawmapboundary(fill_color=fillcol)
map.fillcontinents(color=contcol,lake_color=lakecol)
map.drawcoastlines()
map.readshapefile('Bezirke__Berlin', 'berlin')
#map.readshapefile('gis_osm_roads_free_1', 'berlin')

poptim = poptim.drop_duplicates('lat')    
poptim.apply(plotmarks, axis=1)
    
plt.show()

# Gif

In [ ]:
os.chdir(pdir+"/Populartimes/data")
poptim = pd.read_csv("berlin.csv")
plotgif(poptim)